# 「シゴトがはかどるPython自動処理の教科書」 クジラ飛行机 著 (つづき)
## Excelの高度な作業も自動化しよう

**Excelテンプレートを活用しよう**

In [3]:
# テンプレートから請求書を作成
template_file = 'excel_chap3/invoice-template.xlsx'

# 保存ファイル
save_file = f'excel_chap3/imvoice_{name}.xlsx'

# 設定するデータ
name = '田中一郎'
subject = '１月分のご請求'
items = [ #内訳
['りんご', 5, 320],
['バナナ', 8, 210],
['メロン', 1, 1200]
]

import openpyxl as xl
wb = xl.load_workbook(template_file)
ws = wb.active

ws['b4'] = name
ws['c10'] = subject
total = 0
for i, it in enumerate(items):
    summary, count, price = it
    subtotal = count * price
    total += subtotal
    row_ = 15 + i
    ws.cell(row_, 2, summary)
    ws.cell(row_, 5, count)
    ws.cell(row_, 6, price)
    ws.cell(row_, 7, subtotal)
ws['c11'] = total

wb.save(save_file)


In [4]:
# enumerateの動作確認
food = ['ラーメン', '寿司', '天ぷら']
for i, v in enumerate(food):
    print(i, '=', v)

0 = ラーメン
1 = 寿司
2 = 天ぷら


**複数ファイル処理--売上データから請求書を作成しよう**  
globモジュールの使い方

**Step1. 複数のExcelファイルを読み込む**

In [20]:
# globモジュールの動作確認
import glob
files = glob.glob('excel_chap3/*.xlsx')
print(files)
files_2 = glob.glob('*.*')
print(files_2)

['excel_chap3\\imvoice_田中一郎.xlsx', 'excel_chap3\\invoice-template.xlsx']
['check.bat', 'check.py', 'Python_Jidousyorino_Kyoukasyo_1.ipynb', 'Python_Jidousyorino_Kyoukasyo_2.ipynb', 'README.md']


In [24]:
import openpyxl as xl
import glob

target_dir = 'excel_chap3/salesbooks/'
save_file = 'excel_chap3/matome.xlsx'

def read_files():
    wb = xl.Workbook()
    main_sheet = wb.active
    enumfiles(main_sheet)
    wb.save(save_file)

def enumfiles(main_sheet):
    files = glob.glob(target_dir + '*.xlsx')
    print(files)
    for fname in files:
        read_book(main_sheet, fname)

def read_book(main_sheet, fname):
    print('read:',fname)
    wb = xl.load_workbook(fname, data_only=True)
    ws = wb.active
    rows = ws['a4:f999']
    for row in rows:
        values = [cell.value for cell in row]
        if values[0] is None:
            break
        print(values)
        main_sheet.append(values)

if __name__ == '__main__':
    read_files()


['excel_chap3/salesbooks\\sales-inoue.xlsx', 'excel_chap3/salesbooks\\sales-kakei.xlsx', 'excel_chap3/salesbooks\\sales-ooda.xlsx', 'excel_chap3/salesbooks\\sales-tanaka.xlsx', 'excel_chap3/salesbooks\\sales-yamda.xlsx']
read: excel_chap3/salesbooks\sales-inoue.xlsx
[datetime.datetime(2020, 3, 1, 0, 0), '小島 重吉', 'ミカン', 2, 2500, 5000]
[datetime.datetime(2020, 3, 2, 0, 0), '伊藤 次郎', 'リンゴ', 1, 1500, 1500]
[datetime.datetime(2020, 3, 3, 0, 0), '西川 三郎', 'ミカン', 3, 2500, 7500]
[datetime.datetime(2020, 3, 15, 0, 0), '山田 史朗', 'ミカン', 4, 2500, 10000]
[datetime.datetime(2020, 3, 16, 0, 0), '小島 重吉', 'リンゴ', 2, 1500, 3000]
[datetime.datetime(2020, 3, 17, 0, 0), '山田 史朗', 'ブドウ', 2, 3000, 6000]
[datetime.datetime(2020, 3, 22, 0, 0), '小島 重吉', 'ブドウ', 2, 3000, 6000]
read: excel_chap3/salesbooks\sales-kakei.xlsx
[datetime.datetime(2020, 3, 6, 0, 0), '上田 祐輝', '醤油', 20, 1500, 30000]
[datetime.datetime(2020, 3, 11, 0, 0), '松山 竜兵', 'ソース', 2, 1500, 3000]
[datetime.datetime(2020, 3, 20, 0, 0), '上田 祐輝', '味噌', 8, 95

**Step2. データを顧客ごとに分けて集計しよう**

In [26]:
dummy_data = [
    ['伊藤', 300],
    ['伊藤', 600],
    ['伊藤', 200],
    ['田中', 300],
    ['田中', 200]
]
# 顧客名をキーに辞書型に分割
users = {}
for row in dummy_data:
    name, value = row # 変数に分ける
    # 顧客名が初出ならリストを初期化
    if name not in users:
        users[name] = []
    # 顧客名をキーにして値にデータを追記
    users[name].append(row)
print(users)
# 顧客ごとに集計
for name,rows in users.items():
    print(rows) # 集計対象を表示
    total = 0
    for row in rows:
        total += row[1]
    print(name,total) #結果を表示

{'伊藤': [['伊藤', 300], ['伊藤', 600], ['伊藤', 200]], '田中': [['田中', 300], ['田中', 200]]}
[['伊藤', 300], ['伊藤', 600], ['伊藤', 200]]
伊藤 1100
[['田中', 300], ['田中', 200]]
田中 500


In [29]:
import openpyxl as xl, json

in_file = 'excel_chap3/matome.xlsx'
out_file = 'excel_chap3/matome.json'

def split_list():
    users = read_and_split(in_file)
    result = {}
    for name, rows in users.items():
        result[name] = calc_user(rows)
        print(name, result[name]['total'])
    with open(out_file, 'wt') as fp:
        json.dump(result, fp)

def read_and_split(in_file):
    users = {}
    ws = xl.load_workbook(in_file).active
    for row in ws.iter_rows():
        values = [col.value for col in row]
        name = values[1]
        if name not in users:
            users[name] = []
        users[name].append(values)
    return users

def calc_user(rows):
    total = 0
    items = []
    for row in rows:
        date, _, item, cnt, price, _ = row 
        date_s = date.strftime('%m%d')
        items.append([date_s, item, cnt, price])
        total += cnt * price
    return {'items':items, 'total':total}

if __name__ == '__main__':
    split_list()

小島 重吉 14000
伊藤 次郎 10500
西川 三郎 19100
山田 史朗 34000
上田 祐輝 45600
松山 竜兵 40000
鈴木 太郎 33000
大内 圭人 6000
山下 五郎 5700


**Step3. 顧客ごとに請求書を発行しよう**

In [34]:
import openpyxl as xl, json

in_file = 'excel_chap3/matome.json'
out_dir = 'excel_chap3/invoice02/'
template_file = 'excel_chap3/invoice-template.xlsx'
subject = '２月分のご請求'

# メイン処理
def gen_invoice():
    with open(in_file, 'rt') as fp:
        users = json.load(fp)
    for name, data in users.items():
        make_user_invoice(name, data)

# テンプレートにデータを流し込む
def make_user_invoice(name, data):
    wb = xl.load_workbook(template_file)
    ws = wb.active
    ws['b4'] = name
    ws['c10'] = subject
    ws['c11'] = data['total']
    for i, it in enumerate(data['items']):
        date, summary, cnt, price = it
        row = i +15
        ws.cell(row, 2, summary + '('+date+')')
        ws.cell(row, 5, cnt)
        ws.cell(row, 6, price)
        ws.cell(row, 7, cnt*price)
    out_file = out_dir + name + '様.xlsx'
    wb.save(out_file)
    print('save: ', out_file)

if __name__ == '__main__':
    gen_invoice()

save:  excel_chap3/invoice02/小島 重吉様.xlsx
save:  excel_chap3/invoice02/伊藤 次郎様.xlsx
save:  excel_chap3/invoice02/西川 三郎様.xlsx
save:  excel_chap3/invoice02/山田 史朗様.xlsx
save:  excel_chap3/invoice02/上田 祐輝様.xlsx
save:  excel_chap3/invoice02/松山 竜兵様.xlsx
save:  excel_chap3/invoice02/鈴木 太郎様.xlsx
save:  excel_chap3/invoice02/大内 圭人様.xlsx
save:  excel_chap3/invoice02/山下 五郎様.xlsx


In [35]:
import os
cd = os.getcwd()
print(cd)

C:\Users\sinis\programming\08_Python\Python_Jidousyorino_kyoukasyo


**Excelシートを整形しよう**

**日付形式を西暦から和暦に変更しよう**

In [37]:
wb = xl.load_workbook('excel_chap3/date-sample.xlsx')
ws = wb.active
ws['a4'].number_format

'yyyy"年"m"月"d"日";@'

In [39]:
type(ws['a4'].value)

datetime.datetime

In [66]:
import openpyxl as xl
import datetime

in_file = 'excel_chap3/date-sample.xlsx'
out_file ='excel_chap3/date-wareki.xlsx'
cell_format = 'ggge"年"m"月"d"日"'

def date_wareki(in_file,out_file):
    wb = xl.load_workbook(in_file)
    for ws in wb.worksheets: # 全シートを確認
        for row in ws.iter_rows(): # 全行を確認
            for c in row: #全セルを確認
                check_cell(c)
    wb.save(out_file)

def check_cell(c):
    if type(c.value) == datetime.datetime:
        c.number_format = cell_format


if __name__ == '__main__':
    date_wareki(in_file, out_file)

**条件付き書式を使わずに任意のセルに色をつけよう**

2020年1月1日より前に仕入れた在庫を赤色で強調表示する

In [69]:
import openpyxl as xl
from openpyxl.styles import PatternFill
import datetime

in_file = 'excel_chap3/date-sample.xlsx'
out_file = 'excel_chap3/date-mark_oldtimes.xlsx'
limit = datetime.datetime(2020, 1, 1)

# メイン処理
def date_mark(in_file, out_file):
    wb = xl.load_workbook(in_file)
    for ws in wb.worksheets:
        for r in ws.iter_rows(min_row=4):
            check_row(r)
    wb.save(out_file)
# 指定行をチェック
def check_row(r):
    date = r[0].value
    if type(date) is not datetime.datetime:
        return
    if date < limit:
        red = PatternFill(
            fill_type='solid',
            fgColor='ff0000'
        )
        for c in r:
            c.fill = red

if __name__ == '__main__':
    date_mark(in_file, out_file)

**姓と名に分かれた名前フィールドを一つにまとめよう**

In [70]:
import openpyxl as xl

in_file = 'excel_chap3/name2.xlsx'
out_file = 'excel_chap3/name_conbine.xlsx'

in_wb = xl.load_workbook(in_file)
in_ws = in_wb.worksheets[0]

out_wb = xl.Workbook()
out_ws = out_wb.active

for r in in_ws.iter_rows():
    sei = r[0].value
    na = r[1].value
    seimei = sei + ' ' + na
    out_ws.append([seimei])
out_wb.save(out_file)

**名前フィールドを姓と名に分割しよう**

In [71]:
import openpyxl as xl

in_file = 'excel_chap3/name1.xlsx'
out_file = 'excel_chap3/name_split.xlsx'

wb = xl.load_workbook(in_file)
ws = wb.worksheets[0]
out_wb = xl.Workbook()
out_ws = out_wb.active

for r in ws.iter_rows():
    seimei = r[0].value
    sei, na = seimei.split(' ')
    out_ws.append([sei, na])
out_wb.save(out_file)

**ExcelとWordを連携しよう
-- python-docxのインストール**

Word文書内のテキストを抜き出す

In [1]:
import docx

doc = docx.Document('doc_chap3/letter.docx')
for p in doc.paragraphs:
    print(p.text)

2021年10月10日
●●●御中
■■■様
〒333-0000
東京都新宿区△△△
△△△ビル
△△凸凹
Tel: 03-xxx-xxxx


★★★の送付について


拝啓　ますますご健勝のことと存じます。平素は格別のご厚情にあずかり心より御礼申し上げます。いつも大変お世話になっております。

この度、書類を送付させていただきます。ご多忙の中、恐れ入りますが、どうぞご確認のほど、よろしくお願いします。

敬具



**Wordのテンプレートを活用しよう**

In [2]:
import docx
import datetime

template_file = 'doc_chap3/letter.docx'
save_file = 'doc_chap3/letter-suzuki.docx'
now = datetime.datetime.now()

card ={
    '2021年10月10日': now.strftime('%Y年%m月%d日'),
    '●●●御中': 'マイナビ出版御中',
    '■■■様': '鈴木様',
    '★★★の送付について': '契約書の送付について'
}

doc = docx.Document(template_file)
for p in doc.paragraphs:
    for k, v in card.items():
        if p.text.find(k) >= 0:
            p.text = p.text.replace(k, v)
doc.save(save_file)

**Wordに書式を設定しよう**

In [3]:
import docx
import datetime

template_file = 'doc_chap3/letter.docx'
save_file = 'doc_chap3/letter-suzuki2.docx'
now = datetime.datetime.now()

card ={
    '2021年10月10日': now.strftime('%Y年%m月%d日'),
    '●●●御中': 'マイナビ出版御中',
    '■■■様': '鈴木様',
    '★★★の送付について': '契約書の送付について'
}
cstyle = {
    '★★★の送付について': True
}

doc = docx.Document(template_file)
for p in doc.paragraphs:
    for k, v in card.items():
        if p.text.find(k) >= 0:
            p.text = p.text.replace(k, v)
            if k in cstyle:
                font = p.runs[0].font
                font.bold = True
                font.underline = True
                font.size = docx.shared.Pt(20)
doc.save(save_file)

**ExcelとWordの連携 --顧客名簿を元に案内状を作成**

In [7]:
doc = docx.Document('doc_chap3/template-event.docx')
for p in doc.paragraphs:
    print(p.text)

2021年10月10日
 
 新商品イベントのご案内  
〒333-0000
東京都新宿区△△△△
△△凸凹
Tel: 03-xxx-xxxx

住所：▲▲▲
●●様へ

拝啓　ますますご健勝のことと存じます。平素は格別のご厚情にあずかり心より御礼申し上げます。いつも大変お世話になっております。

今回、下記日程でイベントを開催することになり、案内状を送らせていただきます。素晴らしい商品を取りそろえております。

簡単なプレゼントも用意しています。お客様のご来場、お待ちしております。どうぞ、よろしくお願いします。


敬具



In [9]:
import openpyxl as xl, docx, os

in_file = 'doc_chap3/meibo.xlsx'
template_file = 'doc_chap3/template-event.docx'
save_dir = os.path.join(
    # os.path.dirname(__file__),
    'doc_chap3/',
    'events/'
)

if not os.path.exists(save_dir):
    os.mkdir(save_dir)

def read_book():
    result = []
    ws = xl.load_workbook(in_file).active
    for r in ws.iter_rows(min_row=2):
        v = [c.value for c in r]
        if v[0] is None:
            break
        result.append(v)
    return result

for person in read_book():
    name, zipno, addr = person
    card = {
        '住所：▲▲▲': addr,
        '●●様へ': name + '様へ'
    }
    doc = docx.Document('doc_chap3/template-event.docx')
    for p in doc.paragraphs:
        for k, v in card.items():
            if p.text.find(k) >= 0:
                p.text = p.text.replace(k, v)
                p.runs[0].font.bold = True
    save_file = os.path.join(save_dir,
                             name + '様.docx')
    print('save: ', save_file)
    doc.save(save_file)

save:  doc_chap3/events/松野 厚志様.docx
save:  doc_chap3/events/奥田 仁様.docx
save:  doc_chap3/events/坂本 一郎様.docx
save:  doc_chap3/events/榊原 純一様.docx
save:  doc_chap3/events/黒田 智一様.docx
save:  doc_chap3/events/小倉 義男様.docx
save:  doc_chap3/events/湯浅 良平様.docx
save:  doc_chap3/events/堀 裕二様.docx
save:  doc_chap3/events/清水 源太様.docx
save:  doc_chap3/events/向井 卓哉様.docx
save:  doc_chap3/events/星 康弘様.docx
save:  doc_chap3/events/金子 順吉様.docx
save:  doc_chap3/events/大川 斗真様.docx
save:  doc_chap3/events/関 晋也様.docx
save:  doc_chap3/events/平野 豊様.docx
save:  doc_chap3/events/大西 健介様.docx
save:  doc_chap3/events/花田 健二様.docx
save:  doc_chap3/events/神谷 正様.docx
save:  doc_chap3/events/谷川 隼人様.docx


**CSVとExcelの連携**

In [10]:
import csv, pprint

with open('csv_chap3/items.csv', encoding='sjis') as f:
    text = f.read().strip()
    lines = text.split("\n")
    data = [v.split(',') for v in lines]
    pprint.pprint(data)

[['商品名', ' 値段', ' 個数', ' 小計'],
 ['リンゴ', '300', ' 3', ' 900'],
 ['バナナ', '200', ' 5', ' 1000'],
 ['イチゴ', '800', '2', '1600']]


In [13]:
import csv, pprint

with open('csv_chap3/items.csv', encoding='sjis') as f:
    reader = csv.reader(f)
    data = [row for row in reader]
    pprint.pprint(data)

[['商品名', ' 値段', ' 個数', ' 小計'],
 ['リンゴ', '300', ' 3', ' 900'],
 ['バナナ', '200', ' 5', ' 1000'],
 ['イチゴ', '800', '2', '1600']]


**改行やカンマを含むCSVデータを扱う**

In [15]:
import csv

with open('csv_chap3/items2.csv', encoding='sjis') as f:
    reader = csv.reader(f)
    head = next(reader) #ヘッダ行をスキップ
    total = 0
    for r in reader:
        name, price, cnt, subtotal = r
        print(name,price,cnt,subtotal)
        total += int(subtotal)
    print('合計: ',total, '円')


リンゴ"食べ頃"セット 500 5 1500
果物セットA(バナナ,リンゴ) 1000 5 5000
果物セットB(イチゴ,メロン) 2000 2 4000
合計:  10500 円


**TSVデータを読みたい場合**

In [2]:
import csv

with open('csv_chap3/items3.tsv', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter='\t')
    for row in reader:
        print(row)

['商品名', '値段', '個数', '小計']
['リンゴ"食べ頃"セット', '500', '5', '1500']
['果物セットA(バナナ,リンゴ)', '1000', '5', '5000']
['果物セットB(イチゴ,メロン)', '2000', '2', '4000']


**CSVを読んでExcelシートに書き込もう**

ラベルシート作成

In [4]:
import csv, openpyxl as xl

in_file = 'csv_chap3/meibo.csv'
template_file = 'csv_chap3/card-template.xlsx'
save_file = 'csv_chap3/card-meibo.xlsx'
# csvの読み込み
def read_csv(fname):
    with open(in_file, encoding='sjis') as f:
        reader = csv.reader(f)
        next(reader) #ヘッダを読み飛ばす
        return[v for v in reader]
# Excelの読み込み
wb = xl.load_workbook(template_file)
ws_tp1 = wb.worksheets[0]
# csvから顧客一覧を得て一人ずつ処理
for i, person in enumerate(read_csv(in_file)):
    name, zipno, addr = person
    idx = i % 10
    if idx == 0:
        ws = wb.copy_worksheet(ws_tp1)
        ws.title = 'Page' + str(idx)
    row = 4 * (idx % 5) + 2
    col = 2 * (idx // 5) + 2
    ws.cell(row = row+0, column = col, value = name)
    ws.cell(row = row+1, column = col, value = zipno)
    ws.cell(row = row+2, column = col, value = addr)
# テンプレートのシートを削除して保存
wb.remove(ws_tp1)
wb.save(save_file)

**ExcelファイルをPDFで出力しよう**

pywin32は、操作するソフト（ExcelやWord）が実際にインストールされている必要がある